# publication name, date, byline, section and  headlines

In [1]:
#crawl ['title', 'author', 'time', 'section', 'byline_list', 'url']
## Got Ban From each website , robot detection


import re
import requests
import json
import os


def get_html(url):
    try:
        html = getHTMLText(url)
        return html
    except:
        print("failed connect")


def getHTMLText(url):
    try:
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ''
    
# def crawl_url(url):
#     #download sitemap file
#     sitemap = get_html(url)
#     #extract the sitemap links
#     links = re.findall('<loc>(.*?)</loc>',sitemap)
    
#     return links
    
def reHtml(htmlContent):
    # title
    title = re.findall(r'<h1 class="lede-text-v2__hed">(.*?)</h1>', htmlContent, re.DOTALL)[0].strip()

    # author
    author = re.findall(r'<a class="author-v2__byline".*?>(.*?)</a>', htmlContent, re.DOTALL)[0].strip()

    # time
    time = re.findall(r'<time.*?<noscript>(.*?)</noscript>.*?</time>', htmlContent, re.DOTALL)[0].strip()

    # section
    section = re.findall(r'<div class="eyebrow-v2">(.*?)</div>', htmlContent, re.DOTALL)[0].strip()

    # byline
    byline_list = re.findall(r'<div class="abstract-v2__item-text">(.*?)</div>', htmlContent, re.DOTALL)
    
    return title, author, time, section, byline_list

# Date
def judgeDate(time):
    day = int(re.findall(r'March (\d\d), 2020', time)[0])
    if 18 < day < 26:
        return True
    else:
        return False

def generateData(links):
    '''
    {'NewsTitle': {'title': NewsTitle, 'author': James, 
    'time': 'March 15, 2020, 6:57 PM EDT', 
    'section': 'Business', 'byline_list': ['l1', 'l2']}}
    '''
    data = {}
    keys = ['title', 'author', 'time', 'section', 'byline_list', 'url', 'The']
    
    for link in links:
        htmlContent = get_html(link) 
        print(htmlContent)
        title, author, time, section, byline_list = reHtml(htmlContent)
        info = {}
        info['title'], info['author'], info['time'], info['section'], info['byline_list'], info['url'] = title, author, time, section, byline_list, link
        data[title] = info
    return data

# if __name__ == '__main__':
#     file1 = open('urls.txt','a')
#     file2 = open('htmls.txt','a')
    
#     links = crawl_sitemap('https://www.bloomberg.com/feeds/bbiz/sitemap_news.xml')
    
#     data = generateData(links)
    
#     with open('data.txt', 'w') as outfile:
#         json.dump(data, outfile)


# Crawl only headlines

In [2]:
#crawl only headlines

def crawl_sitemap(url):
    #download sitemap file
    sitemap = get_html(url)
    #extract the sitemap links
    links = re.findall('<loc>(.*?)</loc>',sitemap)
    titles = re.findall('<news:title>(.*?)</news:title>',sitemap)
    times = re.findall('<news:publication_date>(.*?)<news:publication_date>', sitemap)
    return links, titles, times

def crawl_sitemap_Fed(url):
    #download sitemap file
    sitemap = get_html(url)
    #extract the sitemap links
    links = re.findall('<link>(.*?)</link>',sitemap)
    titles = re.findall('<title>(.*?)</title>',sitemap)
    times = re.findall('<pubDate>(.*?)</pubDate>', sitemap)
    return links, titles, times
        

# Prepocessing text and Common words

In [3]:
# Prepocessing text
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
ignore_word = set(['apos','With', 'Says', 'Bloomberg', 'Day', 'Be', 'Is', 'To', 'The', 'Letter', 'business', 'How', 'Amid','new'])

def text_filter(titles):
    #use for word search for title
    dic = {}
    #use for raw text data
    total_words = []
    for s in titles:
        #filter the content in () ex. (article) (radio)/ (podcast)
        s = re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]", "", s)
        #tokenize
        raw_words = nltk.word_tokenize(s)
        wordnet_lematizer = WordNetLemmatizer()
        words = [wordnet_lematizer.lemmatize(raw_word) for raw_word in raw_words]
        #filter
        filtered_words = [word.lower() for word in words if word not in stopwords.words('english') and word.isalpha() and word not in ignore_word]
        total_words += [word for word in filtered_words]
        filtered_words_set = {word for word in filtered_words}
        dic[s] = filtered_words_set
    return total_words, dic

[nltk_data] Downloading package punkt to /Users/siyuqiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/siyuqiu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siyuqiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#most frequent word
from nltk import FreqDist

def most_common(total_words, num):
    freq_dist = FreqDist(total_words) 
    most_common_freq = freq_dist.most_common(num)
    most_common = [word[0] for word in most_common_freq]
    return most_common_freq, most_common

In [5]:
#titles which contain most frequent word
def common_words_with_title(most_common):
    common_word_title_dic = {}
    for word in most_common:
        common_word_title_dic[word] = []
        for k in dic:
            if word in dic[k]:
                common_word_title_dic[word].append(k)
    return common_word_title_dic

In [6]:
# save data files
def save_file(file_name, data):    
    fileObject = open(file_name, 'w')
    for d in data:
        fileObject.write(d)
        fileObject.write('\n')
    fileObject.close()

In [7]:
if __name__ == '__main__':
    # crawl the raw headline data
    links_WSJ, titles_WSJ, times_WSJ = crawl_sitemap('https://www.wsj.com/wsjsitemaps/wsj_google_news.xml')
    links_BB, titles_BB, times_BB = crawl_sitemap('https://www.bloomberg.com/feeds/bbiz/sitemap_news.xml')
    links_FT, titles_FT, times_FT = crawl_sitemap('https://www.ft.com/sitemaps/news.xml')
    links = links_WSJ + links_BB + links_FT
    titles_news = titles_WSJ + titles_BB + titles_FT
    times_news = times_WSJ + times_BB + times_FT
    
    links_fed_press, titles_fed_press, times_fed_press = crawl_sitemap_Fed('https://www.federalreserve.gov/feeds/press_all.xml')
    links_fed_speech, titles_fed_speech, times_fed_speech = crawl_sitemap_Fed('https://www.federalreserve.gov/feeds/speeches.xml')
    titles_fed = titles_fed_press[1:] + titles_fed_speech[1:]
    save_file('fed_data.txt', titles_fed)
        
    # common
    titles = titles_news + titles_fed
    total_words, dic = text_filter(titles)
    most_common_freq, most_common = most_common(total_words, 10)
    common_word_title_dic = common_words_with_title(most_common)
    

In [8]:
most_common_freq

[('coronavirus', 41),
 ('virus', 21),
 ('lockdown', 20),
 ('uk', 19),
 ('federal', 19),
 ('reserve', 17),
 ('us', 15),
 ('board', 15),
 ('bank', 12),
 ('back', 12)]

In [9]:
common_word_title_dic

{'coronavirus': ['Covid Toes a New Symptom of Coronavirus',
  'Disney Takes $1.4 Billion Coronavirus Hit, With Worse to Come',
  'Hong Kong Tests Travelers’ Saliva to Thwart Spread of Coronavirus',
  'When the Coronavirus Outbreak Could Peak in Each U.S. State',
  'France Coronavirus Deaths Rise at Fastest Pace in Six Days',
  'BMW Joins Car Peers With Warning of Coronavirus Profit Slump',
  'U.K. Coronavirus Deaths Pass Italy, With 32,000 Suspected Fatalities',
  'Coronavirus latest: Beijing warns Hong Kong protesters it will not ‘sit idly by’',
  'BDO’s UK staff face fortnightly coronavirus testing ',
  'Coronavirus: return to work divides US meat industry \u2009',
  'Coronavirus threatens €10bn hit to football transfer market',
  'Coronavirus: US deaths spike as fatalities spread outside NY — as it happened',
  'Coronavirus tracked: the latest figures as countries fight to contain the pandemic | Free to read',
  'Disney sets out $1.4bn hit from coronavirus crisis',
  'White House co